In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix, precision_score, recall_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from catboost import CatBoostClassifier

In [8]:
# Загружаем полный обработанный датафрейм
df = pd.read_csv('full_data_new.csv')
df.head()

,id,product,colour,cost,product_sex,base_sale,dt,name_product,category_product,brand,gender,age,education,city,country,personal_coef
0,0,"велосипед горный женский stern mira 20 26""",в нескольких цветах,13599,0,1,7,велосипед,спортинвентарь,stern,0,36,среднее,1201,32,0.5072
1,0,стол outventure,зеленый,1499,2,0,37,стол,отдых/туризм,outventure,0,36,среднее,1201,32,0.5072
2,0,набор outventure стол 4 стула,коричневый,4799,2,0,37,набор,отдых/туризм,outventure,0,36,среднее,1201,32,0.5072
3,3,бутсы мужские gsd astro,белый,1599,1,0,13,бутсы,обувь,другое,1,31,среднее,1134,32,0.4304
4,3,мяч футбольный puma teamfinal 212 fifa quality...,мультицвет,7199,2,0,27,мяч,спортинвентарь,puma,1,31,среднее,1134,32,0.4304


### Модели предсказания склонности клиента принять участие в акции

In [9]:
# Добавим столбец, отображающий была ли в этот день кампания
df['campaign'] = df['dt'].isin([15, 45]).astype(int)

In [6]:
# Датафрейм с участниками второй кампании
train_df = df[df['city'] == 1134]

# Добавим столбец таргет (кто совершил покупку во время акции)
train_df['target'] = train_df.groupby('id')['campaign'].transform('max')

C:\Users\ST\AppData\Local\Temp\ipykernel_14448\3761065785.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['target'] = train_df.groupby('id')['campaign'].transform('max')


In [9]:
# Баланс по таргету
train_df['target'].value_counts(normalize=True)

0    0.664888
1    0.335112
Name: target, dtype: float64

In [20]:
# Удаляем ненужные столбцы
train_df3 = train_df.drop(['id', 'product', 'city', 'country', 'name_product', 'campaign', 'product_sex', 'education', 'colour'], axis =1)
train_df3.head()

,cost,base_sale,dt,category_product,brand,gender,age,personal_coef,target
3,1599,0,13,обувь,другое,1,31,0.4304,0
4,7199,0,27,спортинвентарь,puma,1,31,0.4304,0
5,2799,1,34,обувь,fila,1,31,0.4304,0
6,2999,0,34,обувь,outventure,1,31,0.4304,0
7,9199,0,0,спортинвентарь,другое,0,35,0.5072,0


In [24]:
# Формируем трэйн (до 45 дня акции) и тест (с 45 дня акции) 
train = train_df3[train_df3['dt'] < 45].copy()
test  = train_df3[train_df3['dt'] >= 45].copy()


X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

# Разделяем переменные на числовые и категориальные
num_col = ['cost', 'base_sale', 'gender', 'age', 'personal_coef']
cat_col = ['category_product', 'brand'] 

# Стандартизируем числовые признаки и кодируем категориальные
sc = StandardScaler()
ohe = OneHotEncoder()

prp = ColumnTransformer([
    ('num', sc, num_col),
    ('cat', ohe, cat_col)
])

#### Логистическая регрессия

In [28]:
# Устраняем дисбаланс
smote = SMOTE(random_state=42)

# Модель
lr = LogisticRegression(max_iter=1000, random_state=42)

pl = ImbPipeline([
    ('prep', prp),
    ('smote', smote),
    ('model', lr)
])

# Обучаем
pl.fit(X_train, y_train)

# Предсказания
y_pred = pl.predict(X_test)
y_proba = pl.predict_proba(X_test)[:, 1]

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.5145036911001949
F1-score: 0.44003418478776946
Precision: 0.38938995014284916
Recall: 0.505821568912822
ROC-AUC: 0.5182637665744517


#### Дерево решений

In [26]:
# Модель
rf_c = DecisionTreeClassifier(random_state=42, class_weight='balanced')

pl = ImbPipeline([
    ('prep', prp),
    ('model', rf_c)
])

# Обучаем
pl.fit(X_train, y_train)

# Предсказания
y_pred = pl.predict(X_test)
y_proba = pl.predict_proba(X_test)[:, 1]

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.5820686626965613
F1-score: 0.46105389814913117
Precision: 0.4487771271098863
Recall: 0.4740212487265318
ROC-AUC: 0.5672112532309569


#### Случайный лес

In [30]:
# Устраняем дисбаланс
smote = SMOTE(random_state=42)

# Модель
rf_c = RandomForestClassifier(
    n_estimators=700,
    max_depth=35,
    min_samples_leaf=3,
    min_samples_split=10,
    max_features='sqrt',
    max_samples=0.8,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1)

pl = ImbPipeline([
    ('prep', prp),
    ('model', rf_c)
])

# Обучаем
pl.fit(X_train, y_train)

# Предсказания
y_pred = pl.predict(X_test)
y_proba = pl.predict_proba(X_test)[:, 1]

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.5989461840335903
F1-score: 0.43474897501353754
Precision: 0.464002642007926
Recall: 0.4089652161257459
ROC-AUC: 0.5902922561867107


#### CatBoost

In [33]:
# Устраняем дисбаланс
smote = SMOTE(random_state=42)

# Модель
cat = CatBoostClassifier(
    iterations=3000,
    depth=6,
    learning_rate=0.04,
    l2_leaf_reg=7,
    class_weights=[1, 2],  
    eval_metric='AUC',     
    bagging_temperature=0.3,
    random_strength=1,
    early_stopping_rounds=200,
    verbose=100,
    random_seed=42
)

pl = ImbPipeline([
    ('prep', prp),
    ('smote', smote),
    ('model', cat)
])

# Обучаем
pl.fit(X_train, y_train)

# Предсказание
y_pred = pl.predict(X_test)
y_proba = pl.predict_proba(X_test)[:, 1]

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

0:	total: 14.9ms	remaining: 44.8s
100:	total: 1.13s	remaining: 32.3s
200:	total: 2.27s	remaining: 31.7s
300:	total: 3.43s	remaining: 30.8s
400:	total: 4.54s	remaining: 29.4s
500:	total: 5.69s	remaining: 28.4s
600:	total: 6.86s	remaining: 27.4s
700:	total: 7.98s	remaining: 26.2s
800:	total: 9.12s	remaining: 25s
900:	total: 10.3s	remaining: 23.9s
1000:	total: 11.5s	remaining: 23s
1100:	total: 12.7s	remaining: 21.9s
1200:	total: 13.8s	remaining: 20.7s
1300:	total: 15s	remaining: 19.6s
1400:	total: 16.2s	remaining: 18.5s
1500:	total: 17.3s	remaining: 17.3s
1600:	total: 18.5s	remaining: 16.1s
1700:	total: 19.6s	remaining: 15s
1800:	total: 20.8s	remaining: 13.8s
1900:	total: 22.1s	remaining: 12.8s
2000:	total: 23.5s	remaining: 11.7s
2100:	total: 24.9s	remaining: 10.7s
2200:	total: 26.4s	remaining: 9.59s
2300:	total: 27.9s	remaining: 8.48s
2400:	total: 29.3s	remaining: 7.32s
2500:	total: 30.8s	remaining: 6.14s
2600:	total: 32.2s	remaining: 4.94s
2700:	total: 33.6s	remaining: 3.72s
2800:	total

#### Многослойный перцептрон

In [36]:
# Устраняем дисбаланс
smote = SMOTE(random_state=42)


# Модель
mlp = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    alpha=1e-4,
    learning_rate_init=0.001,
    max_iter=600,
    early_stopping=True,
    n_iter_no_change=20,
    validation_fraction=0.1,
    random_state=42
)

pl = ImbPipeline([
    ('prep', prp),
    ('smote', smote),
    ('model', mlp)
])

# Обучаем
pl.fit(X_train, y_train)

# Предсказание
y_pred = pl.predict(X_test)
y_proba = pl.predict_proba(X_test)[:, 1]

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))

Accuracy: 0.5392299459370454
F1-score: 0.4581423868843995
Precision: 0.4116214335421016
Recall: 0.5165187017901325
ROC-AUC: 0.5524186695793919


Низкое качество моделей скорее всего связано с небольшим дисбалансом данных, также метрика Accuracy не совсем информативна в данном случае. Возможно, модели недостаточно данных, чтобы предсказать будет ли участвовать клиент в акции или нет. Наилучшей моделью среди представленных является CatBoost, метрика Recall у нее достаточно высокая (ловит 82% целевых клиентов), также неплохое значение F1, но низкое значение Precision (среди предсказанных «1», только 40% настоящие). Точность ниже, но охватывает нужных клиентов. Если у нашей маркетинговой кампании не стоит вопрос в стоимости привлечении каждого клиента (а это так, так как это баннерная реклама, а не персональная рассылка), то можно использовать. 

### Модели предсказания категории товара, которую выберет клиент

In [10]:
train_df6 = df.copy()

In [11]:
# Удаляем ненужные столбцы
train_df6 = train_df6.drop(['id', 'product', 'dt', 'city', 'country', 'campaign', 'name_product'], axis =1)
train_df6.head()

,colour,cost,product_sex,base_sale,category_product,brand,gender,age,education,personal_coef
0,в нескольких цветах,13599,0,1,спортинвентарь,stern,0,36,среднее,0.5072
1,зеленый,1499,2,0,отдых/туризм,outventure,0,36,среднее,0.5072
2,коричневый,4799,2,0,отдых/туризм,outventure,0,36,среднее,0.5072
3,белый,1599,1,0,обувь,другое,1,31,среднее,0.4304
4,мультицвет,7199,2,0,спортинвентарь,puma,1,31,среднее,0.4304


In [12]:
X = train_df6.drop('category_product', axis = 1)
y = train_df6['category_product']

num_col = ['cost', 'product_sex', 'base_sale', 'gender', 'age', 'personal_coef']
cat_col = ['colour', 'education', 'brand'] 

ohe = OneHotEncoder(handle_unknown='ignore')
sc = StandardScaler()

#### Логистическая регрессия

In [9]:
lr = LogisticRegression(max_iter=1000, class_weight='balanced', C=0.5)
prp = ColumnTransformer(transformers = [('num', sc, num_col), ('cat', ohe, cat_col)])
pl = Pipeline(steps=[('prp', prp), ('model', lr)])
y_pred = cross_val_predict(pl, X, y, method='predict')

print("Accuracy:", accuracy_score(y, y_pred))

Accuracy: 0.5958501524386302


#### Дерево решений

In [10]:
# Модели леса
dt_c = DecisionTreeClassifier(random_state=42, class_weight='balanced')
prp = ColumnTransformer(transformers = [('num', sc, num_col), ('cat', ohe, cat_col)])
pl1 = Pipeline(steps=[('prp', prp), ('model', dt_c)])

y_pred1 = cross_val_predict(pl1, X, y)
print("Accuracy:", accuracy_score(y, y_pred1))

Accuracy: 0.8640568069860595


#### Случайный лес

In [13]:
rf_c = RandomForestClassifier(n_estimators=700, random_state=42, max_features = 'sqrt', max_depth=None, class_weight='balanced', min_samples_leaf = 3)
prp = ColumnTransformer(transformers = [('num', sc, num_col), ('cat', ohe, cat_col)])
pl2 = Pipeline(steps=[('prp', prp), ('model', rf_c)])

y_pred2 = cross_val_predict(pl2, X, y)
print("Accuracy:", accuracy_score(y, y_pred2))

Accuracy: 0.8243148387901854


#### Многослойный перцептрон

In [14]:
mlp = MLPClassifier(
    max_iter=600, 
    hidden_layer_sizes=(100,10), 
    activation = 'relu',
    early_stopping=True,
    n_iter_no_change=20,
    learning_rate_init=0.001,
    solver='adam',
    alpha=1e-4
)
pl3 = Pipeline(steps=[('prp', prp), ('model', mlp)])

y_pred3 = cross_val_predict(pl3, X, y)
print("Accuracy:", accuracy_score(y, y_pred3))

Accuracy: 0.7635030442943529


Наилучшими моделями в данном случае оказались модели дерева решений и случайный лес. Это можно объяснить тем, что такие модели хорошо работают с нелинейными зависимостями и категориальными переменными и не требуют сложного тюнинга. \
Данные по продуктам были достаточно размазаны и разнообразны. Признак category_product был создан вручную на основании текстового описания товара. Нельзя точно сказать, что категории были максимально разделены правильно, но это в любом случае упростило работу модели. Также нужно учесть, что один и тот же клиент может купить совершенно разные товары, поэтому сложно полностью предсказать его поведение, поэтому данные метрики являются неплохими для данной задачи.
